In [294]:
import numpy as np
import pycristoforo as pyc
import requests
import itertools
import geopy.distance

In [295]:
nodes = []
n_nodes = 10
n_agents = 4

In [296]:
n_suppliers = 3
n_buyers = n_nodes - n_suppliers

In [297]:
nodes_attributes = ["id", "x", "y", "type", "products"]

In [298]:
products = [("banana", 0, 10000), ("orange", 0, 10000), ("strawbery", 0, 1000), 
            ("skip", 0, 500), ("meat", 0, 600), ("fish", 0, 400), ("spinach", 0, 3000)]

In [299]:
enough_resources_world = True

In [300]:
prod = [p[0] for p in products]
tots = [0 for i in products]

for id_ in range(n_suppliers):
    y,x= get_coordinates("PORTUGAL")
    
    n_diff_products = np.random.randint(len(products))+1
    products_to_use = np.random.choice(np.arange(len(products)), n_diff_products, False)
    products_for_sale = {products[i][0]:np.random.randint(products[i][1],products[i][2]) for i in products_to_use}
    
    for p in products_for_sale:
        index = prod.index(p)
        tots[index] = tots[index]+products_for_sale[p]
    
    a = {"id":id_,"x":x,"y":y, "type":"supplier","products":products_for_sale}
    nodes.append(a)

In [301]:
for id_ in range(n_suppliers, n_nodes):
    y,x= get_coordinates("PORTUGAL")
    
    n_diff_products = np.random.randint(len(products))+1
    if enough_resources_world:
        if sum(tots)<10:
            break
        products_to_use = [i for i,v in enumerate(tots) if v > 0]
        products_to_use = np.random.choice(products_to_use, min(len(products_to_use),n_diff_products), False)
        products_ordered = {products[i][0]:np.random.randint(products[i][1],tots[i]) for i in products_to_use}
        for i in products_ordered:
            tots[prod.index(i)] = tots[prod.index(i)] - products_ordered[i]
    else:
        products_to_use = np.random.choice(np.arange(len(products)), n_diff_products, False)
        products_ordered = {products[i][0]:np.random.randint(products[i][1],products[i][2]) for i in products_to_use}
    
    for p in products_for_sale:
        index = prod.index(p)
        tots[index] = tots[index]+products_for_sale[p]
    
    a = {"id":id_,"x":x,"y":y, "type":"buyer","products":products_ordered}
    nodes.append(a)

In [302]:
edges = []
for i,j in itertools.combinations(np.arange(len(nodes)), 2):
    #compute distance
    coords_1 = (nodes[i]['x'], nodes[i]['y'])
    coords_2 = (nodes[j]['x'], nodes[j]['y'])
    dist = geopy.distance.distance(coords_1, coords_2).km
    #create edge
    edge = {"node_1":nodes[i]['id'], "node_2":nodes[j]['id'], 'distance':dist}
    edges.append(edge)

In [303]:
map_ = {"nodes":nodes, 'edges':edges, "directed": False, "no_agents":n_agents}

In [304]:
map_

{'nodes': [{'id': 0,
   'x': -8.061410745460432,
   'y': 39.933077277432425,
   'type': 'supplier',
   'products': {'strawbery': 140,
    'fish': 264,
    'meat': 290,
    'spinach': 1274,
    'skip': 197,
    'orange': 1536,
    'banana': 9771}},
  {'id': 1,
   'x': -8.554587731686716,
   'y': 41.076596734966856,
   'type': 'supplier',
   'products': {'strawbery': 417,
    'orange': 6579,
    'spinach': 1609,
    'fish': 161,
    'meat': 537,
    'skip': 315,
    'banana': 1971}},
  {'id': 2,
   'x': -7.688011168498097,
   'y': 40.126252071330356,
   'type': 'supplier',
   'products': {'meat': 591,
    'strawbery': 860,
    'banana': 7984,
    'fish': 375,
    'skip': 419,
    'orange': 616}},
  {'id': 3,
   'x': -8.56716854244296,
   'y': 41.15498350308525,
   'type': 'buyer',
   'products': {'orange': 673, 'skip': 398}},
  {'id': 4,
   'x': -8.561308658738561,
   'y': 41.45134964201477,
   'type': 'buyer',
   'products': {'banana': 12376,
    'fish': 1026,
    'meat': 661,
    'stra

In [305]:
import json

with open('map_commerce1.json', 'w') as fp:
    json.dump(map_, fp)

In [251]:
f = open("C:\\Users\\Pedro\\Documents\\MultiAgentMarket\\keys\\maps_ppreto_key.txt", "r")
key = f.read()
##prepare request string and make request
points_call = "|".join([str(i['x']) + "," + str(i['y']) for i in nodes])
dm = requests.get(
    "https://maps.googleapis.com/maps/api/distancematrix/json?origins={ori}&destinations={dest}&key={key}".format(
        ori=points_call, dest=points_call, key=key))
## put it in the required format
dm_dict = {'response': dm.json, 'points_ids': [i['id'] for i in nodes], 'points_coor': [(i['x'], i['y']) for i in nodes], 'size': len(nodes)}

In [ ]:
def get_coordinates(country):
    shape = pyc.get_shape(country)
    point = pyc.geoloc_generation(shape, 1, "PORTUGAL")
    return [(p['geometry']['coordinates'][1], p['geometry']['coordinates'][0]) for p in point][0]